In [2]:
import numpy as np

In [ ]:
def lu_decomposition_inplace(A, dU=None):
    n = A.shape[0]
    
    # Creăm o copie a lui A pentru a nu modifica matricea originală
    A_copy = A.copy()
    
    if dU is None:
        dU = np.ones(n)
    
    # Verificăm dacă primul element este zero
    if A_copy[0, 0] == 0 or dU[0] == 0:
        raise ValueError("am gasit un minor cu det=0, deci nu se poate continua cu LU")
    
    # Inițializăm primul element al lui L (salvat în diagonala principală)
    A_copy[0, 0] = A_copy[0, 0] / dU[0]
    
    # Calculăm prima linie de U (elementele de deasupra diagonalei principale)
    for j in range(1, n):
        A_copy[0, j] = A_copy[0, j] / A_copy[0, 0]
    
    # Calculăm prima coloană de L (elementele de sub diagonala principală)
    for i in range(1, n):
        A_copy[i, 0] = A_copy[i, 0] / dU[0]
    
    # Procesăm restul matricei
    for p in range(1, n):
        # Calculăm L[p, p]
        sum_l = 0
        for k in range(p):
            sum_l += A_copy[p, k] * A_copy[k, p] * dU[k]
        
        if A_copy[p, p] - sum_l == 0 or dU[p] == 0:
            raise ValueError(f"am gasit un minor cu det=0 (pasul {p}), deci nu se poate continua cu LU")
        
        A_copy[p, p] = (A_copy[p, p] - sum_l) / dU[p]
        
        # Calculăm rândul p din U
        for j in range(p+1, n):
            sum_u = 0
            for k in range(p):
                sum_u += A_copy[p, k] * A_copy[k, j] * dU[k]
            
            A_copy[p, j] = (A_copy[p, j] - sum_u) / (A_copy[p, p] * dU[p])
        
        # Calculăm coloana p din L
        for i in range(p+1, n):
            sum_l = 0
            for k in range(p):
                sum_l += A_copy[i, k] * A_copy[k, p] * dU[k]
            
            A_copy[i, p] = (A_copy[i, p] - sum_l) / dU[p]
    
    return A_copy, dU

# After the decomposition, A = LU and det(A) = det(L) * det(U) 
# Therefore det(A) = det(L) * det(U) = product of the diagonal elements of U and L
def find_determinant(A, dU):
    return np.prod([A[i][i] for i in range(A.shape[0])]) * np.prod([dU[i] for i in range(len(dU))])


# Given A = LU we have to solve the system
# Ax = b.
# Since A = LU we can solve for
# LUx = b

# Using substitution we can substitute Ux as y , therefore we will need to solve 2 triangular systems

# Ly = b
# and 
# Ux = y

# Then we modify to solve the system with the inplace solution of the LU 
def solve_system(A, dU, b):
    n = A.shape[0]
    
    # Ly = b (fofward substitution)
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i]
        for j in range(i):
            y[i] -= A[i, j] * y[j]
        y[i] /= A[i, i]
    
    # Ux = y (backward substitution)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for j in range(i+1, n):
            x[i] -= A[i, j] * x[j]
        x[i] /= dU[i]
    
    return x

# Solve using numpy
def solve_system_numpy(A, b):
    return np.linalg.solve(A, b)

# Matrix inverse using numpy
def matrix_inverse(A):
    return np.linalg.inv(A)

def difference_of_solutions(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# Verify the solution using the norm of the difference between the product of A and x minus b
def verify_solution(A, x, b):
    return difference_of_solutions(np.dot(A, x), b) < 1e-9


In [ ]:
import numpy as np

A = np.array([
    [4, 2, 3],
    [2, 7, 5.5],
    [6, 3, 12.5]
])

b = [21.6, 33.6, 51.6]

dU = [2, 3, 4]


# A = np.array([
#     [2.5, 2, 2],
#     [-5, -2, -3],
#     [5, 6, 6.5]
# ])

# b = [2, -6, 2]

# dU = [1, 1, 1]



ALU, newDu = lu_decomposition_inplace(A, dU)
print("LU Decomposition of A =\n", ALU)

x = solve_system(ALU, newDu, b)
xNumPy = solve_system_numpy(A, b)
Ainv = matrix_inverse(A)
print("Determinant using LU decomposition:\n", find_determinant(ALU, newDu))
print("Solution to the system Ax = b\n x =", x)
print("The solution is valid:", verify_solution(A, x, b))
print("Solution to the system Ax = b using solely numpy\n x =", xNumPy)
print("A inverse =\n", Ainv) 
print("||x-xlib|| :\n", difference_of_solutions(x,xNumPy))
print("||x-Ainv*b|| :\n", difference_of_solutions(x,np.dot(Ainv,b)))
    


LU Decomposition of A =
 [[ 2.          1.          1.5       ]
 [ 1.          1.66666667  0.5       ]
 [ 3.         -1.          1.25      ]]
Determinant using LU decomposition:
 100.0
Solution to the system Ax = b
 x = [-1.264  3.464  6.576]
The solution is valid: False
Solution to the system Ax = b using solely numpy
 x = [2.5 2.2 2.4]
A inverse =
 [[ 0.36979167 -0.08333333 -0.05208333]
 [ 0.04166667  0.16666667 -0.08333333]
 [-0.1875     -0.          0.125     ]]
||x-xlib|| :
 5.762323142622251
||x-Ainv*b|| :
 5.762323142622251
